In [1]:

from variables import sourcepath
import os
import pandas as pd
import numpy as np
import etl.dim_author as auth
import etl.dim_journal as jour
import etl.common_functions as cof
import etl.dim_paper as pape
import etl.database as db
from db_credentials import dwh_db_connection_params
import datetime

In [4]:
eng, psycop2connect=db.initialize_engine(connection_params=dwh_db_connection_params)

In [2]:
source_facts=cof.load_sourcefile('entities.csv')[['sentence_id', 'ent_id', 'entity']]
source_facts

,sentence_id,ent_id,entity
0,1_331_363,knowledge repository,knowledge repository
1,1_476_511,digital platform,platform strategy
2,1_511_536,Salesforce.com,Salesforce.com
3,1_539_560,participatory design,user participation
4,1_874_899,group participant,two groups
...,...,...,...
2137613,7484_13503_13545,government,Government
2137614,7484_13503_13545,health information system,Health information technology
2137615,7484_13545_13568,America,America
2137616,7484_13545_13568,Slovenia,Slovenia


In [11]:
source_facts['new']='MISSING'
source_facts=source_facts.assign(new2='MISSING')

In [12]:
source_facts

,sentence_id,ent_id,entity,new,new2
0,1_331_363,knowledge repository,knowledge repository,MISSING,MISSING
1,1_476_511,digital platform,platform strategy,MISSING,MISSING
2,1_511_536,Salesforce.com,Salesforce.com,MISSING,MISSING
3,1_539_560,participatory design,user participation,MISSING,MISSING
4,1_874_899,group participant,two groups,MISSING,MISSING
...,...,...,...,...,...
2137613,7484_13503_13545,government,Government,MISSING,MISSING
2137614,7484_13503_13545,health information system,Health information technology,MISSING,MISSING
2137615,7484_13545_13568,America,America,MISSING,MISSING
2137616,7484_13545_13568,Slovenia,Slovenia,MISSING,MISSING


In [4]:
print(source_facts.shape)
#tbd introduce count fact measure? currently duplicates get dropped but maybe this should be fixed!
source_facts.drop_duplicates().shape


(2137618, 3)


(1997939, 3)

In [9]:
#source_facts.drop_duplicates(inplace=True)
#source_facts=
source_facts.groupby(['sentence_id', 'ent_id']).transform('size')
#.size().reset_index().rename(columns={0:'entity_count', 'entity': 'entity_instance'}).shape

/tmp/ipykernel_4171634/2240191065.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the transforming function.
  source_facts.groupby(['sentence_id', 'ent_id']).transform('size')


TypeError: Transform function invalid for data types

In [10]:
dim_sentence=db.load_full_table(eng, 'dim_sentence')
dim_entity=db.load_full_table(eng, 'dim_entity')

In [15]:
dim_entity.head(2)

,entity_pk,entity_label,entity_name
0,1,TOPIC,knowledge repository
1,2,TOPIC,digital platform


In [16]:
source_facts=pd.merge(source_facts, dim_sentence, how='left', left_on='sentence_id', right_on='sentence_source_id')[['ent_id', 'sentence_pk']]
source_facts=pd.merge(source_facts, dim_entity, how='left', left_on='ent_id', right_on='entity_name')[['entity_pk', 'sentence_pk']]
source_facts

,entity_pk,sentence_pk
0,1,16.0
1,2,20.0
2,3,21.0
3,4,22.0
4,5,33.0
...,...,...
1935577,181,3324087.0
1935578,186,3324087.0
1935579,256,3324088.0
1935580,987,3324088.0
